In [9]:
#handling data
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter


In [10]:
# Read json into a pandas dataframe
tweets_df = pd.read_json("insurance.txt", lines=True)

In [11]:
tweets_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'extended_tweet', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang',
       'timestamp_ms', 'display_text_range', 'retweeted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'extended_entities'],
      dtype='object')

In [126]:
all_hashtags = []
for e in tweets_df.entities:
    hashtags = [t['text'] for t in e['hashtags']]
    all_hashtags = all_hashtags + hashtags

from collections import Counter
c = Counter(all_hashtags)
most_common_tuples = c.most_common()
sorted_keys = sorted(c, key=c.get, reverse=True)
most_common_tuples

[('fintech', 231),
 ('AI', 152),
 ('insurance', 96),
 ('BigData', 67),
 ('IoT', 59),
 ('Fintech', 53),
 ('Insurance', 45),
 ('COVID19', 42),
 ('insurtech', 42),
 ('finserv', 38),
 ('legal', 38),
 ('ArtificialIntelligence', 37),
 ('FinTech', 34),
 ('MachineLearning', 34),
 ('DataScience', 33),
 ('Blockchain', 30),
 ('goldenmoneyteam', 28),
 ('blockchain', 27),
 ('digital', 27),
 ('agodgmt', 27),
 ('Analytics', 26),
 ('CyberSecurity', 25),
 ('banking', 25),
 ('DigitalTransformation', 24),
 ('innovation', 23),
 ('ai', 23),
 ('Bitcoin', 22),
 ('DataScientist', 22),
 ('MarketScreener', 21),
 ('payments', 19),
 ('DeepLearning', 19),
 ('Insurtech', 18),
 ('startup', 18),
 ('Goldenmoneyteam', 18),
 ('Spotify', 18),
 ('law', 17),
 ('EmergingTechnologies', 15),
 ('robot', 15),
 ('Pandemic', 15),
 ('crypto', 14),
 ('HealthTech', 13),
 ('Innovation', 13),
 ('ML', 12),
 ('claim', 12),
 ('finance', 11),
 ('VR', 11),
 ('technology', 11),
 ('Cloud', 11),
 ('robotics', 11),
 ('regtech', 10),
 ('cryptoc

In [127]:
for t in most_common_tuples:
    tweets_df['#'+t[0]] = False

In [128]:
# tweets_df.at[0, '#AppleMusic'] = False

In [129]:
for i, e in enumerate(tweets_df.entities):
    hashtags = [t['text'] for t in e['hashtags']]
    for hashtag in hashtags:
        tweets_df.at[i, '#' + hashtag] = True
    

In [130]:
for t in tweets_df[tweets_df['#insurance'] == True]['text'].values:
    print(t)

Here's a list of common reasons that a #homeowners #insurance policy can be cancelled.  https://t.co/pVITkFIpPV https://t.co/fbsEIdlFEf
RT @PLAYER1Rising: Pretend everyone was rich @SpeakerPelosi or had an #insurance policy protecting them retroactively during a national cri…
RT @floodtweeter: Even if your property is not in a high-risk #flood area, you should consider flood #insurance. Learn more about how #FEMA…
RT @globaliqx: Why data and #AI will become the new normal in #insurance

https://t.co/rBOFO3bho9 

#BigData  #insurtech 

@labordeolivier…
RT @globaliqx: Why data and #AI will become the new normal in #insurance

https://t.co/rBOFO3bho9 

#BigData  #insurtech 

@labordeolivier…
Our #Chiropractors work with many #insurance companies. Do you know whether you are covered? https://t.co/1MpS1krOc3
RT @ESlaw_London: #covid19 and upcoming thought leadership ...
#financialservices #banking #insurance #Assetmanagement #payments #PrivateEq…
Did you know the cost of your #homeowners #i

# Getting media

In [131]:
media_list = list(filter(lambda x: isinstance(x, dict), tweets_df['extended_entities'].values.tolist()))
for media in media_list:
    for m in media['media']:
        print(m['type'], m['media_url'])

photo http://pbs.twimg.com/media/EYAfLwdX0AYvcOG.jpg
photo http://pbs.twimg.com/media/EYAfXCsXgAg_SZr.jpg
photo http://pbs.twimg.com/media/EYAmZrxXsAUV-2e.jpg
photo http://pbs.twimg.com/media/EYAma8wWsAAQQP1.jpg
photo http://pbs.twimg.com/media/EYAohJsWkAAuR3c.jpg
photo http://pbs.twimg.com/media/EYAo0FIX0AMQkNT.jpg
photo http://pbs.twimg.com/media/EX-wCr4WkAgl-8D.jpg
photo http://pbs.twimg.com/media/EYAtL5iXQAA1TOQ.jpg
photo http://pbs.twimg.com/media/EYAuVjQWsAIKmjd.jpg
video http://pbs.twimg.com/ext_tw_video_thumb/1251037581908860931/pu/img/OsaUJKWsK6Glo1Qx.jpg
photo http://pbs.twimg.com/media/EYAwsLMWAAEQwJ8.jpg
video http://pbs.twimg.com/ext_tw_video_thumb/1260946764548489220/pu/img/cXocYxCzWBzes3vX.jpg
photo http://pbs.twimg.com/media/EX-60FDVcA0aSGz.jpg
photo http://pbs.twimg.com/media/EYAypKLWsAE-kxE.jpg
photo http://pbs.twimg.com/media/EYA3zsxU0AAOBGm.jpg
photo http://pbs.twimg.com/media/EYA5NWjVcAcjVPP.jpg
photo http://pbs.twimg.com/media/EYA-PXiXYAE8pX8.jpg
photo http://pbs.

# Full text

In [132]:
import math
def apply_func(x):
    if not isinstance(x, float):
        if 'full_text' in x: 
            return x['full_text']
        else:
            return float('nan')
    else:
        return float('nan')
    

tweets_df['full_text'] = tweets_df['extended_tweet'].apply(lambda x: apply_func(x))
tweets_df.full_text.fillna(tweets_df.text, inplace=True)

# Links

In [14]:
import re 
from collections import Counter
import requests

import extraction
import requests



url_list = []
for text in tweets_df['text'].values.tolist():
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    url_list += urls

    

url_list = list(filter(lambda url: len(url)>13, url_list))

c = Counter(url_list)
most_common_urls = c.most_common()
sorted_urls = sorted(c, key=c.get, reverse=True)




for url in sorted_urls:
    resp = requests.head(url)
    try:
        url = resp.headers["Location"]
        html = requests.get(url).text
        extracted = extraction.Extractor().extract(html, source_url=url)
        print(extracted.title, extracted.description, extracted.image, url)
        print('\n')
    except:
        print(url)
        print('\n')

x_url = 'https://twitter.com/i/web/status/1259958047033548800'
web_status_string = "https://twitter.com/i/web/status/"
x_url.startswith(web_status_string)

urbebank.com steht zum Verkauf - Sedo GmbH Keine Daten verfügbar! https://cdn.sedo.com/images/visa.png http://urbebank.com/


Why data and machine intelligence will become the new normal in insurance | Swiss Re Machine learning and artificial intelligence are the latest tools that purport to improve this pre-emptive ability and make better decisions. https://www.swissre.com/dam/jcr:aa95fb32-db21-430b-89d2-72b640be5fc0/data-and-machine-intelligencel-in-insurance.jpg https://buff.ly/3bmZxAI


Four Artificial Intelligence Technologies to Lead the Global Economy Out of the Pandemic Technology innovation in artificial intelligence (AI) is accelerating at a breakneck pace, and the ability to innovate, adopt and integrate AI techniques to evolve business models will separate those businesses that recover from the COVID-19 pandemic from those that will fail. https://www.ipwatchdog.com/wp-content/uploads/2020/05/Depositphotos_207844888_xl-2015-scaled.jpg https://buff.ly/2Z3myWM


R Programming 

Charles Hoskinson Will Ethereum ETH2 make Cardano obsolete Charles Hoskinson Will Ethereum ETH2 make Cardano obsolete Full Video with no editing here: https://youtu.be/rpNe1qkjnz4 https://i.ytimg.com/vi/hA1CLEGvZgM/maxresdefault.jpg https://youtu.be/hA1CLEGvZgM


403 Forbidden None None http://cognitivesoftware.com/


Bloomberg - Are you a robot? To continue, please click the box below to let us know you're not a robot. None https://www.bloomberg.com/news/articles/2020-05-13/how-citigroup-morgan-stanley-are-returning-to-asia-offices


Malicious Use of AI Poses a Real Cybersecurity Threat We should prepare for a future in which artificially intelligent cyberattacks become more common. https://twimgs.com/nojitter/darkreading/dr-logo.jpg https://j.mp/3fpHtJi


Jeff Bezos ‘Trillionaire’ Is Trending On Twitter. Here’s Why Bezos has a long way to go to become the world’s first trillionaire. https://d1epq84pwgteub.cloudfront.net/f9b5a66da80e419082d159ec6563e965.jpg http://on.forbesmiddleeast.

IBM claims its Neural Computer achieves record AI model training time IBM claims its Neural Computer can train an AI algorithm to beat Atari 2600 games in record time -- and perform on par with state-of-the-art approaches. https://venturebeat.com/wp-content/uploads/2019/11/ibm-keyboard-logo.png?w=1200&strip=all https://buff.ly/3cAAaww


Indian-origin engineer charged in US with over $10 million COVID-relief fraud New York, May 14 (PTI) An Indian-origin engineer in the US has been charged with fraudulently seeking more than USD 10 million in loans under a coronavirus relief programme set up to help small businesses. https://www.outlookindia.com/public/uploads/outlook-banner-img.gif http://ow.ly/yPcG50zGERw


Who is responsible for digital transformation in insurance? For many business executives, digital transformation has lost its meaning. Its definition seems to change depending on the situation. That doesn't mean organizations can ignore it. Instead, businesses need to make sure that

How AI can help us harness our 'collective intelligence' Collaborating usually makes our work better – but keeping a team on task is not easy. Now, researchers are finding that machines can bring out the best in group work. https://ichef.bbci.co.uk/wwfeatures/live/624_351/images/live/p0/8d/07/p08d0715.jpg https://buff.ly/2T4d0H9


Facebook Offers $50K For AI That Can Detect Hateful Memes Facebook's created a dataset of 10,000 ‘hateful’ memes and is offering prizes totaling $100,000 for developers who can use it to detect hate speech in memes. https://thumbor.forbes.com/thumbor/fit-in/1200x0/filters%3Aformat%28jpg%29/https%3A%2F%2Fspecials-images.forbesimg.com%2Fimageserve%2F5e58eaf07a009800073473b9%2F0x0.jpg https://buff.ly/35YQsgw


15th May - Law News LawNewsIndex is a UK based legal news & articles archive since 2011 focusing on Law, Lawyers, Law Firms, Justice, Legislation, Litigation, Legal Ethics, Human Rights & Social Justice issues. https://lh4.googleusercontent.com/proxy/8cK_y

fooo.ooo on Twitter “[🔴 NEWS] How to Rent a Camper https://t.co/eLm0AunZyo #Insurance #RV #RecreationalVehicle #MinorLeagueBaseball #Rent #FinalDestination #PickupTruck #Driveway #Beach #SouthwesternUnitedStates #Summit #PrimeRate #VehicleInsurance #Camping #CommercialDriversLicense” https://pbs.twimg.com/profile_images/1211342040522534912/xIG_HQSt_400x400.jpg https://twitter.com/i/web/status/1261041373089869827


AMERICAN US INSURANCE on Twitter “Cyber Liability is the risk that is posed by businesses when they store information on the internet or in paper files. . #amerinsured #cyber #liability #insurance #cyberliability #usinsurance #bocaraton #hacking #breach #damageexpense #mitigation #security #tech #cybercrime” https://pbs.twimg.com/media/EYAeolmX0AQkymE.jpg:large https://twitter.com/i/web/status/1261041591231426560


Back to Work California! on Twitter “@TheHartford @TheHartford_PR @CDInews @ICRicardoLara @EPIC_Insurance no plan to offer #Discounts to #shutdown businesses in #C

...

Rob Morris on Twitter “Dormant Insight #bigdata #analytics #law #court #litigation #data #change #futurist #Gladwell argues seeing defendant not helpful https://t.co/zaFyG1tcm0” https://pbs.twimg.com/profile_images/1174012492961521665/5Lc2asW4_400x400.jpg https://twitter.com/i/web/status/1261114097640210432


Bernard Moon on Twitter “SparkLabs Global Ventures Promotes Jessica Jackley, Co-Founder of Kiva, to General Partner https://t.co/eGXtMqOorm via @TechCrunch Jessica is an inspirational #entrepreneur that will be strengthening our team and firm as a whole #VentureCapital #founders #fintech #sustainability” https://pbs.twimg.com/media/EYBgl5cWsAUL5Bb.jpg:large https://twitter.com/i/web/status/1261114101209731073


ISB Insurance Inc on Twitter “Here’s a handy tip! When you pick a #Medicare supplement #insurance policy, you’ll want to go with a plan that will serve you well today &amp; 15-20 years from now. https://t.co/4kBy5p3EMU” https://pbs.twimg.com/profile_images/10989739541302313

Twitter / ? You can search Twitter using the search box below or return to the homepage . https://abs.twimg.com/errors/logo23x19.png https://twitter.com/i/web/status/1261116895375196161


BIconnections on Twitter “How to learn #datascience &amp; Machine learning? During this surreal time, we found ourselves with some more time in our hands if some of us are pl .. https://t.co/SEajPAZyA7 #fintech #BigData #algorithms https://t.co/hspFyunt1D” https://pbs.twimg.com/profile_images/1866862979/50519_388209157862059_947380685_n_400x400.jpg https://twitter.com/i/web/status/1261116896210030592


Brinc on Twitter “*Brinc Portfolio* - @lastbitpay building a truly next-generation hardware and software solution to securely store and manage your cryptocurrency: https://t.co/4a0MV06qkT #Crypto #Bitcoin #Cryptocurrency #Hardware #BitcoinWallet #CryptoWallet #HardwareWallet #Blockchain #FinTech” https://pbs.twimg.com/media/EYBjSrcU8AMTH5K.png:large https://twitter.com/i/web/status/1261117122240901120



Rafael Gonzalez, Esq. on Twitter “Between March 1st and May 2nd, 2020, more than 31 million people filed for unemployment insurance. In addition to loss of income, job loss carries the risk of loss of health insurance https://t.co/ppXxYdEZEM #rafaelgonzalezesq #unemployed #healthinsurance” https://pbs.twimg.com/profile_images/1012160303507726336/b827mGjo_400x400.jpg https://twitter.com/i/web/status/1261120091929837570


FintechZoom on Twitter “New article: "Paying down debt in the coronavirus economic downturn" has been published on Fintech Zoom - https://t.co/6CsD19KOiX @FintechZoom #Fintech #Blockchain #Crypto #Cryptocurrency #ICO” https://pbs.twimg.com/profile_images/1059887179508473856/716o6otx_400x400.jpg https://twitter.com/i/web/status/1261120119226368001


Tim Chan on Twitter “#Insurers and service providers have more time to prepare for Royal Commission reforms but some provisions have been fast tracked https://t.co/fW5J5nrzfq @InsuranceBizAU” https://pbs.twimg.com/profile_ima

Temasek Joins Libra Association — Facebook's Global Digital Currency Project - Fintech Singapore The Libra Association, an independent member organization, is pleased to announce the addition of new members, Temasek. https://fintechnews.sg/wp-content/uploads/2020/05/The-Libra-Association-Announces-New-Members.jpg https://buff.ly/2y2t591


Fintech Singapore on Twitter “. @Temasek Joins Libra Association — Facebook’s Global Digital Currency Project https://t.co/J9OrdghiNp #fintech #digitalcurrency #blockchain #digitaltransformation” https://pbs.twimg.com/media/EYBoTFIXYAEM1gb.jpg:large https://twitter.com/i/web/status/1261122574454607872


Cady Darago on Twitter “Interesting. I'm still waiting to see how/if the legal industry adapts to new ways of doing business. https://t.co/xt4U6KXUiv #esquisitemarketing #LegalMarketing #legal #Lawyers #lawfirm” https://pbs.twimg.com/profile_images/1131747751442694150/U4FVwhTL_400x400.png https://twitter.com/i/web/status/1261122675465994242


Twitter /

True

Apache Kafka Installation on Mac using Homebrew
Hi all,
https://miro.medium.com/max/612/1*tsXkwb5Q57qUzUhC07ESFQ.png
